# **BPTI-KUNITZ DOMAIN**

The aim of this project is building an HMM of the Kunitz-BPTI domain starting from structural information. This HMM to be then validated against SwissProt.

### **PDB SEARCH**

The first step is to retrieve all the available 3d structures carrying a PF00014 domain.

PDB advanced search:
-resolution <= 3.5 angstroms AND PF00014



## **PDBe FOLD**

The chain I of 3TGI known as a PF00014 has been ran against the list of IDs (from the previous pdb search)  in PDBe FOLD to retrieve the portion of the PF00014 sequence for all these stuctures.


The cluster of representative sequences has been generated using CD-HIT to remove redundancy from the PDBe fold result:


In [1]:
!cd-hit -i PDBe_FOLD_result  -o cluster.txt

Program: CD-HIT, V4.8.1, Mar 01 2019, 06:12:48
Command: cd-hit -i PDBe_FOLD_result -o cluster.txt

Started: Sun May  3 09:09:05 2020
                            Output                              
----------------------------------------------------------------
total seq: 206
longest and shortest : 65 and 52
Total letters: 11718
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 75M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90513177

comparing sequences from          0  to        206

      206  finished         13  clusters

Approximated maximum memory consumption: 75M
writing new database
writing clustering information
program completed !

Total CPU time 0.05


From this file the IDs and the chains have been retrieved with:

In [ ]:
!cat cluster.txt|grep ">"|awk 'BEGIN {FS=OFS=":"} {print $2, $3}'| cut -d " " -f 1 > Cluster_ids.txt

Then a multiple structure alignments with PDBe fold has been performed on this set.
The sequence alignment derived from structural alignment has been downloaded and used to generate the HMM model using HAMMER 

In [2]:
!hmmbuild Bpti-kunitz.hmm PDBe-fold_seq_ali.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             PDBe-fold_seq_ali.txt
# output HMM file:                  Bpti-kunitz.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     PDBe-fold_seq_ali       13    69    58     1.89  0.960 

# CPU time: 0.03u 0.00s 00:00:00.03 Elapsed: 00:00:00.03


The logo from the resulting HMM model has been generated with skylign

# **Filtering of seed sequences from the positive set¶**

The PF00014 set has been downloaded from Uniprot.
To remove the sequences used to build the model.
I can do it by blasting the positives against the curated seed used to build the model. 

In [3]:
!makeblastdb -in cluster.txt -dbtype prot




Building a new DB, current time: 05/03/2020 09:10:10
New DB name:   /Users/danielelucarelli/Desktop/desktop/bioinformatic /unibo/2 semestre/lab1/capriotti/data_rep/def/cluster.txt
New DB title:  cluster.txt
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/danielelucarelli/Desktop/desktop/bioinformatic /unibo/2 semestre/lab1/capriotti/data_rep/def/cluster.txt
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 13 sequences in 0.00409913 seconds.




In [4]:
!psiblast -query PF00014.fasta -db cluster.txt -outfmt 6 -out blast_PF00014vscluster.txt


**data filtering**
From the blast result the IDs of all the sequences with a percent identity more than 95 and an alignment coverage shorter than 52 have been retrieved and excluded from the PF00014 fasta file.

In [ ]:
!awk '{if($3<=95 && $4>52) split($1,v,"|"); print v[2]}' blast_PF00014vscluster.txt|sort -u > Blast_ID_list.txt

In [ ]:
! ./scripts/get_blast_id_retrieve-sequences.py Blast_ID_list.txt PF00014.fasta POSITIVE_nocluster.txt 1

## ** TRAIN & TEST **

All the not PF00014 proteins has been downloaded from Uniprot and they will be used as negative set.

**data preparation**
An **hmm search** of the profile against the sequences has been generated in the 2 subsets


In [ ]:
!hmmsearch --cpu 6 --noali --max -Z 1 --tblout positive_set.txt Bpti-kunitz.hmm POSITIVE_nocluster.txt

In [ ]:
!hmmsearch --cpu 8 --noali --max -Z 1  --tblout NEGATIVE_set.txt Bpti-kunitz.hmm total_uniprot_notPF00014.fasta

**data extraction**
A new tab separated file with each protein ID, the best domain e-value and a label (0 negatives,1 positives) has been created for each set.

In [ ]:
! awk '!/^#/ {split($1,v,"|"); print v[2],$8,1}' positive_set.txt|sort -gk 2 >OK_positive_set.txt

In [ ]:
! awk '!/^#/ {split($1,v,"|"); print v[2],$8,0}' NEGATIVE_set.txt|sort -gk 2 >OK_negative_set.txt

For the negative set the sequences which has not passed trought the filtering step of the HMM search has been exluded. In order to reintegrate them i will add an e-value of 11 by default to them (1 point above the max e-value used by default by HMM search)

In [ ]:
!comm <(awk '/^>/{split($1,a,"|"); print a[2]}' <total_uniprot_notPF00014.fasta |sort) <(awk '{print$1}' OK_negative_set.txt|sort)|awk -F '\t' '{if ($1!="") print $1,11,0}'>excluded_negatives.txt

Now all the sets can be merged

In [ ]:
! cat OK_positive_set.txt OK_negative_set.txt excluded_negatives.txt > SET.txt

In [9]:
! grep '1$' SET.txt|wc

     335    1005    5599


Just to check that anything has worked properly

And converted to csv format

In [10]:
!python3 ./scripts/convert_to_csv.py SET.txt SET.csv

The following scrpit perform a stratified **k-fold cross validation** of the method dividing the dataset in three subsets.

In [5]:
!python3 ./scripts/TrainingHMM.py SET.csv


Splitting the dataset  3 times
1e-20 0.9999288113090354 0.9383730228764705
1e-19 0.9999483881990507 0.9557170400557664
1e-18 0.9999679650890659 0.9727525394490515
1e-17 0.9999786433927106 0.9819203476011275
1e-16 0.9999822028272589 0.9849573650077618
1e-15 0.999983982544533 0.9864723757009461
1e-14 0.9999875419790812 0.9894954546210867
1e-13 0.9999928811309036 0.994012875058743
1e-12 0.9999928811309036 0.994012875058743
1e-11 0.9999928811309036 0.994012875058743
1e-10 0.9999928811309036 0.994012875058743
1e-09 0.9999964405654518 0.9970131445941504
1e-08 0.9999946608481777 0.9955274968095258
1e-07 0.9999928811309036 0.9940484652909726
1e-06 0.9999875419790812 0.989650581142807
1e-05 0.9999786433927106 0.9824486267631763
0.0001 0.9999608462199695 0.9686787843654876
0.001 0.9996992277806748 0.815157961610284
0.01 0.9968410018384479 0.39782650264524805
0.1 0.9581535077337614 0.11601344178493521
1 0.7059729091436534 0.03779239616656858
Fold  1
selected threshold 1e-09
    Training     	Tes

## The best threshold is 1e-09 with only one false positve and one false negative